In [6]:
import numpy as np
import pandas as pd
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=11a5787cbb54d48fd5555f0dca731f24fbf689c479bb28849903611af8a76e03
  Stored in directory: /Users/sgkang09/Library/Caches/pip/wheels/46/78/0e/8e5e2b500f83a682c8d7e7ce820638cf99faa894a662f71cf0
Successfully built wget


## Install `libaarhusxyz`

Available at: https://github.com/emerald-geomodelling/libaarhusxyz

In [46]:
import wget
import libaarhusxyz

## Download data from Stanford Digital Library

In [8]:
url = 'https://stacks.stanford.edu/file/druid:yc041mz9691/skytem-kaweah-final.csv'
wget.download(url)

  9% [.....                                               ]  1646592 / 16977766

 16% [........                                            ]  2768896 / 16977766

 26% [.............                                       ]  4505600 / 16977766

 33% [.................                                   ]  5619712 / 16977766

 42% [......................                              ]  7208960 / 16977766

...

 72% [.....................................               ] 12238848 / 16977766

 80% [.........................................           ] 13664256 / 16977766

 94% [................................................    ] 15974400 / 16977766

100% [....................................................] 16977766 / 16977766

'skytem-kaweah-final.csv'

In [9]:
url = 'https://stacks.stanford.edu/file/druid:yc041mz9691/20181205_456_Keweah_312_DualWaveform_60Hz_skb_SR2.gex'
wget.download(url)

100% [............................................................] 6053 / 6053

'20181205_456_Keweah_312_DualWaveform_60Hz_skb_SR2.gex'

## Read SkyTEM data and .gex file

In [38]:
df_kaweah = pd.read_csv("skytem-kaweah-final.csv")

In [40]:
waveform_io = libaarhusxyz.gex
system_kaweah = waveform_io.parse("20181205_456_Keweah_312_DualWaveform_60Hz_skb_SR2.gex")

header [General] parsed
header [Channel1] parsed
header [Channel2] parsed


## Digest (?) .gex file 

In [41]:
# SkyTEM kaweah
unit_conversion = 1e-12
area = system_kaweah['General']['TxLoopArea']

i_start_hm = 10
i_start_lm = 10
i_end_hm = 10 + 27
i_end_lm = 10 + 20 -2 

waveform_hm_kaweah = system_kaweah['General']['WaveformHMPoint']
waveform_lm_kaweah = system_kaweah['General']['WaveformLMPoint']
time_input_currents_HM_kaweah = waveform_hm_kaweah[:,0] 
input_currents_HM_kaweah = waveform_hm_kaweah[:,1]
time_input_currents_LM_kaweah = waveform_lm_kaweah[:,0] 
input_currents_LM_kaweah = waveform_lm_kaweah[:,1]

time_gates = system_kaweah['General']['GateTime']
GateTimeShift=system_kaweah['Channel1']['GateTimeShift']
MeaTimeDelay=system_kaweah['Channel1']['MeaTimeDelay']
NoGates=int(system_kaweah['Channel1']['NoGates'])
t0_lm_kaweah = waveform_lm_kaweah[:,0].max()
times_lm_kaweah = (time_gates[:NoGates,0] + GateTimeShift + MeaTimeDelay)[i_start_lm:i_end_lm]

GateTimeShift=system_kaweah['Channel2']['GateTimeShift']
MeaTimeDelay=system_kaweah['Channel2']['MeaTimeDelay']
NoGates=int(system_kaweah['Channel2']['NoGates'])
t0_hm_kaweah = waveform_hm_kaweah[:,0].max()
times_hm_kaweah = (time_gates[:NoGates,0] + GateTimeShift + MeaTimeDelay)[i_start_hm:i_end_hm]    

In [42]:
lm_inds = np.array(['DBDT_Ch1' in key.split('GT') for key in df_kaweah.keys()], dtype=bool)
hm_inds = np.array(['DBDT_Ch2' in key.split('GT') for key in df_kaweah.keys()], dtype=bool)
lm_header = (df_kaweah.keys()[lm_inds])[i_start_lm:i_end_lm]
hm_header = (df_kaweah.keys()[hm_inds])[i_start_hm:i_end_hm]

## Print active channel names and corresponding time channels

In [44]:
lm_header

Index(['DBDT_Ch1GT11', 'DBDT_Ch1GT12', 'DBDT_Ch1GT13', 'DBDT_Ch1GT14',
       'DBDT_Ch1GT15', 'DBDT_Ch1GT16', 'DBDT_Ch1GT17', 'DBDT_Ch1GT18',
       'DBDT_Ch1GT19', 'DBDT_Ch1GT20', 'DBDT_Ch1GT21', 'DBDT_Ch1GT22',
       'DBDT_Ch1GT23', 'DBDT_Ch1GT24', 'DBDT_Ch1GT25', 'DBDT_Ch1GT26',
       'DBDT_Ch1GT27', 'DBDT_Ch1GT28'],
      dtype='object')

In [47]:
times_lm_kaweah

array([2.821e-05, 3.522e-05, 4.421e-05, 5.571e-05, 7.021e-05, 8.821e-05,
       1.107e-04, 1.387e-04, 1.742e-04, 2.197e-04, 2.767e-04, 3.487e-04,
       4.397e-04, 5.537e-04, 6.977e-04, 8.792e-04, 1.108e-03, 1.396e-03])

In [45]:
hm_header

Index(['DBDT_Ch2GT11', 'DBDT_Ch2GT12', 'DBDT_Ch2GT13', 'DBDT_Ch2GT14',
       'DBDT_Ch2GT15', 'DBDT_Ch2GT16', 'DBDT_Ch2GT17', 'DBDT_Ch2GT18',
       'DBDT_Ch2GT19', 'DBDT_Ch2GT20', 'DBDT_Ch2GT21', 'DBDT_Ch2GT22',
       'DBDT_Ch2GT23', 'DBDT_Ch2GT24', 'DBDT_Ch2GT25', 'DBDT_Ch2GT26',
       'DBDT_Ch2GT27', 'DBDT_Ch2GT28', 'DBDT_Ch2GT29', 'DBDT_Ch2GT30',
       'DBDT_Ch2GT31', 'DBDT_Ch2GT32', 'DBDT_Ch2GT33', 'DBDT_Ch2GT34',
       'DBDT_Ch2GT35', 'DBDT_Ch2GT36', 'DBDT_Ch2GT37'],
      dtype='object')

In [48]:
times_hm_kaweah

array([0.00037671, 0.00038372, 0.00039271, 0.00040421, 0.00041871,
       0.00043671, 0.0004592 , 0.0004872 , 0.0005227 , 0.0005682 ,
       0.0006252 , 0.0006972 , 0.0007882 , 0.0009022 , 0.0010462 ,
       0.0012277 , 0.0014565 , 0.0017445 , 0.0021085 , 0.0025675 ,
       0.0031455 , 0.0038645 , 0.0047445 , 0.0058215 , 0.0071395 ,
       0.0087535 , 0.0107285 ])